In [102]:
import psycopg2
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [103]:
## connect to db
conn = psycopg2.connect( host="database-1.ctqg0gsq67st.eu-north-1.rds.amazonaws.com",
                         database="postgres", user="postgres", password="javamylife", port="5432")
cursor = conn.cursor()

In [104]:
## extract dataset into a df
query = """
select currenttype.title as currenttype, usagetype.title as usage, connectiontype.title as connection_type, addressinfo.stateorprovince as state, addressinfo.town as town, connection.powerkw as power_kw, connection.quantity as number_connections 
from openchargemap 
inner join addressinfo 
on openchargemap.addressid = addressinfo.addressid 
inner join usagetype
on usagetype.usagetypeid = openchargemap.usagetypeid 
inner join connection 
on connection.connectionid = openchargemap.connectionid 
inner join connectiontype 
on connectiontype.connectiontypeid = connection.connectiontypeid
inner join currenttype
on connection.currenttypeid = currenttype.currenttypeid"""

df = pd.read_sql(query, conn)

conn.close()

/var/folders/6g/4zq1d985075_7_l6gxj2n81m0000gn/T/ipykernel_50889/1890283615.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [105]:
df

,currenttype,level,usage,connection_type,state,town,power_kw,number_connections
0,DC,Level 3: High (Over 40kW),Public,CHAdeMO,Wisconsin,Oak Creek,50.0,1.0
1,DC,Level 3: High (Over 40kW),Public,CHAdeMO,Missisipi,Ridgeland,63.0,2.0
2,DC,Level 3: High (Over 40kW),Public,CHAdeMO,Alabama,Guin,80.0,4.0
3,DC,Level 3: High (Over 40kW),Public - Membership Required,CCS (Type 1),AR,Bryant,180.0,4.0
4,DC,Level 3: High (Over 40kW),Public,CCS (Type 1),Alabama,Athens,400.0,6.0
...,...,...,...,...,...,...,...,...
59587,AC (Single-Phase),Level 2 : Medium (Over 2kW),Public,Type 1 (J1772),CA,Pasadena,4.0,2.0
59588,None,Level 1 : Low (Under 2kW),Private - Restricted Access,Unknown,CA,Santa Rosa,NaN,2.0
59589,AC (Single-Phase),Level 2 : Medium (Over 2kW),Private - Restricted Access,Type 1 (J1772),CA,Sacramento,4.0,9.0
59590,AC (Single-Phase),Level 2 : Medium (Over 2kW),Private - Restricted Access,Type 1 (J1772),CA,Los Angeles,4.0,338.0


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59592 entries, 0 to 59591
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   currenttype         56515 non-null  object 
 1   level               59135 non-null  object 
 2   usage               58050 non-null  object 
 3   connection_type     59592 non-null  object 
 4   state               59573 non-null  object 
 5   town                59567 non-null  object 
 6   power_kw            56546 non-null  float64
 7   number_connections  57280 non-null  float64
dtypes: float64(2), object(6)
memory usage: 3.6+ MB


Encoding Columns

In [107]:
## label encode categorical columns
def label_encode_columns(df, column_names):

    df_encoded = df.copy()
    encoder = LabelEncoder()

    for col in column_names:
        if col in df_encoded.columns:
            df_encoded[col] = encoder.fit_transform(df_encoded[col])
    return df_encoded

cate_cols = ['currenttype', 'usage', 'connection_type', 'state', 'town', 'level']

df = label_encode_columns(df, cate_cols)

In [108]:
df = df.dropna()

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


X = df.drop(columns=['power_kw'])
y = df['power_kw']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

model = RandomForestRegressor(n_estimators = 20, max_features = 'sqrt', max_depth = 10, random_state = 42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

rmse = mse**.5
print("Root Mean Squared Error:", rmse)

r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

Mean Squared Error: 345.3541278884234
Root Mean Squared Error: 18.583705978313997
R-squared score: 0.8605570460475707
